In [49]:
import google.generativeai as genai

import os
from dotenv import load_dotenv
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex,StorageContext,load_index_from_storage
from IPython.display import display, Markdown
from llama_index.embeddings.gemini import GeminiEmbedding




In [50]:
load_dotenv()

api_key=os.getenv("GEMINI_API_KEY")

if api_key=="":
 print("api key not found")
else:
 print("api key found")

api key found


In [51]:
genai.configure(api_key=api_key)
for models in genai.list_models():
    print(models.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental
models/embedding-001
models/text-embedding-004
models/aqa
models/imagen-3.

In [63]:
documents=SimpleDirectoryReader("../QueansRAG/text")

In [65]:
import nbconvert
doc=documents.load_data()

In [66]:
print(doc[0].text)

What is machine learning?
Machine learning (ML) is a branch of artificial intelligence (AI) focused on enabling computers and machines to imitate the way that humans learn, to perform tasks autonomously, and to improve their performance and accuracy through experience and exposure to more data.
A Decision Process: In general, machine learning algorithms are used to make a prediction or classification. Based on some input data, which can be labeled or unlabeled, your algorithm will produce an estimate about a pattern in the data.

An Error Function: An error function evaluates the prediction of the model. If there are known examples, an error function can make a comparison to assess the accuracy of the model.

A Model Optimization Process: If the model can fit better to the data points in the training set, then weights are adjusted to reduce the discrepancy between the known example and the model estimate. The algorithm will repeat this iterative “evaluate and optimize” process, updatin

In [67]:
from llama_index.core.settings import Settings
from llama_index.core.node_parser import TokenTextSplitter

from llama_index.llms.gemini import Gemini

llm = Gemini(model="models/gemini-1.5-pro-latest", api_key=api_key)

gemini_embed_model = GeminiEmbedding(model_name="models/embedding-001", api_key=api_key)

Settings.llm = llm
Settings.embed_model = gemini_embed_model

text_splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=20)

chunks = text_splitter.split_text(doc[0].text)
print(f"Number of Chunks: {len(chunks)}")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk}")

for doc in doc:
    embedding = gemini_embed_model.get_text_embedding(doc.text)
    print(f"Embedding size: {len(embedding)}")


Number of Chunks: 8
Chunk 1: What is machine learning?
Machine learning (ML) is a branch of artificial intelligence (AI) focused on enabling computers and machines to imitate the way that humans learn, to perform tasks autonomously, and to improve their performance and accuracy through experience and exposure to more data.
A Decision Process: In general, machine learning algorithms are used to make a prediction or classification. Based on some input data, which can be labeled or unlabeled, your algorithm will produce an estimate about a pattern in the
Chunk 2: which can be labeled or unlabeled, your algorithm will produce an estimate about a pattern in the data.

An Error Function: An error function evaluates the prediction of the model. If there are known examples, an error function can make a comparison to assess the accuracy of the model.

A Model Optimization Process: If the model can fit better to the data points in the training set, then weights are adjusted to reduce the discrep

In [70]:
index=VectorStoreIndex.from_documents(documents=[doc],text_splitter=text_splitter)
# print(f"Indexed {len(index.docstore.docs)} documents.")
storage_context = StorageContext.from_defaults(persist_dir="./storage")


In [71]:
index.storage_context.persist()

In [72]:
query_engine = index.as_query_engine(retrieval_mode="similarity", similarity_top_k=3)
response=query_engine.query("What is the document about?")

In [73]:
print(response.response)

This document explains machine learning (ML) as a branch of artificial intelligence (AI) that allows computers to learn like humans.  It describes the core components of ML algorithms: a decision process, an error function, and a model optimization process.  The document also differentiates between machine learning, deep learning, and neural networks, explaining how these concepts relate to each other and how they function.  It highlights the differences in learning approaches between "deep" and "non-deep" machine learning, and clarifies the structure and operation of neural networks, including the significance of layers in deep learning.

